<p style="font-family:courier;font-size:300%;color:#184d47;background-color:#e4d3cf;">Importing Packages</p>

In [ ]:
import pandas as pd
import numpy as np
from colorama import Fore as f
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
import re
import gc
from warnings import filterwarnings
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import clear_output
import plotly.express as px
from nltk.corpus import stopwords as st
import nltk
from nltk.corpus import webtext
from nltk.probability import FreqDist
from collections import deque
import plotly
plotly.offline.init_notebook_mode (connected = True)


filterwarnings("ignore")

<p style="font-family:courier;font-size:300%;color:#184d47;background-color:#e4d3cf;">Importing Data</p>

In [ ]:
data=pd.read_csv('../input/wikibooks-dataset/english-wikibooks/en-books-dataset.csv')

In [ ]:
data.head()

<p style="font-family:courier;font-size:300%;background-color:#e4d3cf;">What is this data all about ??</p>
<ul style="font-family:courier;font-size:200%;"><li>Title : Title of the wikibook </li>
    <li>Url : Link to the wiki book </li>
    <li>Abstract : A Summary of the wiki book </li>
    <li>Body text : Content of the wiki book </li>
    <li> Body html : Html code of the wiki book </li>

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/2011Sendai-NOAA-Energylhvpd9-05.jpg/1280px-2011Sendai-NOAA-Energylhvpd9-05.jpg)

In [ ]:
title=data['title'][200]
link=data['url'][200]
abstract=data['abstract'][200]
body=data['body_text'][200]
print(f.YELLOW+"Title : ",f.CYAN,title)
print(f.YELLOW+"Url : ",f.CYAN,link)
print(f.YELLOW+"Abstract : ",f.CYAN,abstract)
print(f.YELLOW+"Body : ",f.CYAN,' '.join(body.split('.')[:2]))

<p style="font-family:courier;font-size:300%;background-color:#e4d3cf;">Wordcloud For The Text Body</p>

In [ ]:
stopwords=list(STOPWORDS)
# Generate a word cloud image
data['body_text']=data['body_text'].apply(lambda x : str(x))
text=' '.join(data['body_text'].loc[:1000])
mask = np.array(Image.open("../input/wiki-logo/Wikipedia-logo_%28inverse%29.png"))
arrow=np.array(Image.open("../input/arrowww/8iAj9Xbia.png"))
arrow=cv2.resize(arrow,(300,300))
another_image=np.array(Image.open('../input/multicoloredflower/Multi coloured flower.jpg'))
mask=cv2.resize(mask,(600,600))
another_image=cv2.resize(another_image,(600,600))
wordcloud_fra = WordCloud(stopwords=stopwords, background_color="white", mode="RGBA", max_words=1000, mask=mask).generate(text)
fig, ax = plt.subplots(1,3,figsize=(18,8),gridspec_kw={'width_ratios': [3, 1,3]})
ax[0].imshow(mask)
ax[0].set_axis_off()

ax[1].imshow(arrow)
ax[1].set_axis_off()
# create coloring from image
image_colors = ImageColorGenerator(another_image)
ax[2].imshow(wordcloud_fra.recolor(color_func=image_colors), interpolation="bilinear")
ax[2].set_axis_off()
# store to file
plt.savefig("wiki.png", format="png")
del text
gc.collect()

<p style="font-size:80%">wikipedia logo from https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/Wikipedia-logo_%28inverse%29.png/986px-Wikipedia-logo_%28inverse%29.png</p>

<p style="font-family:courier;font-size:300%;background-color:#e4d3cf;">Top N Words Of The Data</p>

In [ ]:
tex=' '.join(data['title']).lower()
stop_words = set(st.words('english')) 
tex=re.sub(r'(?is)[^a-zA-Z0-9 ]','',tex)
tex=re.sub(r'^[^ ]*', '', tex)
with open("/usr/share/nltk_data/corpora/webtext/Output.txt", "w") as text_file:
    text_file.write(tex)
nltk.download('webtext')
wt_words = webtext.words('./Output.txt')
data_analysis = nltk.FreqDist(wt_words)
# Let's take the specific words only if their frequency is greater than 3.
for i in stop_words :
    del data_analysis[i]
data_analysis = {k: v for k, v in sorted(data_analysis.items(), key=lambda item: item[1],reverse=True)}
key=list(data_analysis.keys())[:100]
item=list(data_analysis.values())[:100]

<p style="font-family:courier;font-size:150%;background-color:#e4d3cf;">Would personally recommend running the animation ( Idea taken from solar system) </p>

In [ ]:
df=pd.DataFrame(columns=['Words','Count','x','y'])
radius=12
to_append=[key[0],item[0]/40,0,0]
a_series = pd.Series(to_append, index = df.columns)
df = df.append(a_series, ignore_index=True)
count=1
for j in range(4) :
    x=radius*j/4
    y=radius**2-x**2
    to_append = [key[count],item[count],x,y]
    a_series = pd.Series(to_append, index = df.columns)
    df = df.append(a_series, ignore_index=True)
    count+=1
for j in range(4):
    x=radius-radius*j/4
    y=-1*(radius**2-x**2)
    to_append = [key[count],item[count],x,y]
    a_series = pd.Series(to_append, index = df.columns)
    df = df.append(a_series, ignore_index=True)
    count+=1
for j in range(4):
    x=-1*(radius*j/4)
    y=-1*(radius**2-x**2)
    to_append = [key[count],item[count],x,y]
    a_series = pd.Series(to_append, index = df.columns)
    df = df.append(a_series, ignore_index=True)
    count+=1
for j in range(4):
    x=-radius+(radius*j/4)
    y=radius**2-x**2
    to_append = [key[count],item[count],x,y]
    a_series = pd.Series(to_append, index = df.columns)
    df = df.append(a_series, ignore_index=True)
    count+=1
    
def rotate_n(df,number):
    x=[]
    p=df.iloc[1:].values
    for i in range(len(p)):
        x.append([p[i,2],p[i,3]])
    t=x.copy()
    time=1
    ho=len(x)
    times=[time]*ho
    for i in range(number-1):
        time+=1
        t = deque(t)
        t.rotate(-1)
        t = list(t)
        x.extend(t)
        times.extend([time]*ho)
    x=np.array(x)
    dn=pd.DataFrame()
    words=[]
    count=[]
    for i in range(len(p)):
        words.append(p[i,0])
        count.append(p[i,1])
    words=words*number
    count=count*number
    dn['Words']=words
    dn['Count']=count
    dn['x']=x[:,0]
    dn['y']=x[:,1]
    dn['Time']=times
    for i in range(1,len(set(times))+1):
        to_append = [df['Words'][0],df['Count'][0],df['x'][0],df['y'][0],i]
        a_series = pd.Series(to_append, index = dn.columns)
        dn = dn.append(a_series, ignore_index=True)
        
    return dn
df=rotate_n(df,20)

        
# for i in range(0,len(key)-3,3):
#     for j in range(k):
#         x=radius*j/k
#         y=radius**2-x**2
#         to_append = [key[i],item[i],x,y,time]
#         a_series = pd.Series(to_append, index = df.columns)
#         df = df.append(a_series, ignore_index=True)
#         y2=0
#         to_append = [key[i+1],item[i+1],x,y2,time]
#         a_series = pd.Series(to_append, index = df.columns)
#         df = df.append(a_series, ignore_index=True)
#         y3=-1*(radius**2-x**2)
#         to_append = [key[i+1],item[i+1],x,y3,time]
#         a_series = pd.Series(to_append, index = df.columns)
#         df = df.append(a_series, ignore_index=True)
#     time+=1
        
# df['Count']=df['Count'].apply(lambda x : int(x))
fig=px.scatter(df,x='x',y='y',text='Words',size='Count',size_max=40,color='Count',
               color_continuous_scale='solar',labels={'x':'','y':''},template='plotly_dark',animation_frame='Time')
fig.add_layout_image(
        dict(
            source="https://upload.wikimedia.org/wikipedia/en/thumb/8/80/Wikipedia-logo-v2.svg/1024px-Wikipedia-logo-v2.svg.png",
            xref="x",
            yref="y",
            x=-8,
            y=40,
            sizex=30,
            sizey=105,
            opacity=0.5,
            layer="below")
)
fig.add_layout_image(
        dict(
            source="https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/Wikipedia-logo_%28inverse%29.png/986px-Wikipedia-logo_%28inverse%29.png",
            xref="x",
            yref="y",
            x=2,
            y=40,
            sizex=30,
            sizey=105,
            opacity=0.5,
            layer="below")
)
fig.update_xaxes(range=[-radius-6,radius+6],showgrid=False)
fig.update_yaxes(range=[-radius**2-16,radius**2+16],showgrid=False)
fig.show()

<p style="font-family:courier;font-size:300%;background-color:#e4d3cf;">Search Engine</p>

In [ ]:
# Let's Make a search engine using TFIDF Vectorizer :)
model = TfidfVectorizer(stop_words='english', binary=True, max_features=100_000)
text_embeddings = model.fit_transform(data.title).toarray()

In [ ]:
def search_it(search,print_n_top=10,return_them=False):
    n=print_n_top
    search_embedding=model.transform([search]).toarray()
    areas=np.matmul(text_embeddings,search_embedding.T).T
    k=areas[0]
    k=sorted(k,reverse=True)
    k=k[:n]
    k=set(k)
    idx=[]
    for j in k :
        idx.extend(np.where(areas[0,]==j)[0])
    o=data.iloc[idx].title.values
    l=[]
    count=1
    if len(o)>print_n_top:
        o=o[:print_n_top]
    for i in o :
        print(f.YELLOW,str(count),')',end=' ')
        print(f.CYAN,i)
        l.append(i)
        count+=1
    if return_them :
        return l
         

In [ ]:
ss=input(f.YELLOW+'Search : ')
l=search_it(ss,return_them=True)
s=input(f.YELLOW+"Choose Your option")
s=l[int(s)-1]
clear_output(wait=True)
title=data[data['title']==s]['title'].values[0]
link=data[data['title']==s]['url'].values[0]
abstract=data[data['title']==s]['abstract'].values[0]
body=data[data['title']==s]['body_text'].values[0]
print(f.YELLOW+"Title : ",f.CYAN,title)
print(f.YELLOW+"Url : ",f.CYAN,link)
print(f.YELLOW+"Abstract : ",f.CYAN,abstract)
print(f.YELLOW+"Body : ",f.CYAN,' '.join(body.split('.')[:2]))